<a href="https://colab.research.google.com/github/seungyoonnam/machine-learning/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

앙상블 학습: 정형데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘

- 대다수의 앙상블 학습 알고리즘은 결정 트리를 기반으로 한다.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트
앙상블 학습 중 가장 유명하고 안정적인 성능을 제공
- 여러 개의 결정 트리를 랜덤하게 만들고 각 결정 트리의 예측을 사용해 최종 예측을 출력
- 어떻게 결정 트리를 랜덤하게 만들 것인가?

1. 훈련 데이터를 랜덤하게 구성 -> 훈련세트에 대한 과대적합 문제 해결
  - 입력한 훈련 데이터에서 랜덤하게 샘플 추출(중복 허용)
  - 이렇게 만들어진 샘플을 부트스트랩 샘플(bootstrap sample)이라고 한다.
  - 기본적으로 훈련 세트의 크기와 동일하게 설정

2. 부트스트랩 샘플로 결정 트리를 훈련
  - 각 노드를 분할할 때 전체 특성 중 일부 특성을 무작위로 고른 후 이 중 최선의 분할을 찾음
  - RandomForestClassifier 분류 모델은 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택
  - RandomForestClassifier 회귀 모델은 전체 특성을 사용

3. 정해진 수만큼 위 과정을 반복

랜덤하게 선택된 특성을 사용하므로 훈련세트에 대한 과대적합을 방지할 수 있다.

랜덤 포레스트는 누락값이 있어도 처리가 가능하다

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


oob(Out Of Bag): 부트스트랩 샘플에 포함되지 않은 데이터로 성능테스트

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리
랜덤포레스트와 비슷하게 동작. 단, 훈련세트에서 랜덤하게 샘플을 추출하는 과정이 없이 그대로 전체 훈련세트를 사용한다.
- 노드를 분할할 때 가장 좋은 분할을 찾지 않고 랜덤으로 분할
- 랜덤포레스트보다 더 많은 결정 트리를 훈련해야한다.
- 그러나 랜덤하게 노드를 분할하기 때문에 계산 속도가 빠름

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


1~4장의 모델은 전체 훈련세트 하나에 대해 모델 하나를 적용하는 방식.

랜덤포레스트, 엑스트라포레스트는 훈련세트(부트스트랩샘플, 혹은 전체)에 대해 여러 모델을 적용

지금부터 배우는 부스팅 모델은 sequential하게 훈련세트->학습->훈련세트->학습->... 의 방식으로 진행

## 그레이디언트 부스팅
- 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완
- 사이킷런에서는 default로 깊이3인 결정트리 100개를 사용
- 깊이가 얕은 결정트리를 사용하므로 과대적합에 강하고 일반화 성능이 뛰어나다
- 부스팅이므로 예측 모델을 순차적으로 더해나간 후 최종 모델을 예측
- 즉, 이전 모델의 오류(실제값과 예측값의 차이)를 새로운 모델을 추가하면서 줄이는 것

김성범 고려대 산업공학과 교수 참고자료 youtube 참고

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
#n_estimators: 모델 수. default 100
#learning_rate: 학습률. default 0.1

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅

- 정형데이터 머신러닝 알고리즘 중 가장 인기있는 알고리즘
- 입력 특성을 256 구간으로 나누고 그 중 하나를 떼어놓고 누락된 값을 위해 사용함. 즉, 입력에 누락된 특성이 있어도 따로 전처리할 필요가 없음
히스토그램: 데이터를 일정한 구간으로 쪼개서 막대그래프로 표현
- 입력데이터의 각 특성을 256개의 구간으로 쪼갠다. 즉, 특성 구간을 압축한다.

max_iter: 부스팅 반복 횟수(n_estimator 대신 사용)

n_repeats: 랜덤하게 석을 횟수. default 5

특성 개수*n_repeats 의 값 만큼 학습 수행

In [10]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

현존하는 가장 인기있는 알고리즘이기 때문에 사이킷런 외에도 다양한 라이브러리에서 그래디언트 부스팅을 지원한다.

XGBoost, LightGBM 등

#### XGBoost

In [14]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


#### LightGBM

In [15]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
